In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
chromedriver = 'C:/Users/imkyu/Downloads/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [3]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url = base_url + quote('향수')

In [4]:
driver.get(url)

In [5]:
# blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body  = driver.find_element_by_tag_name('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [7]:
lis = soup.select('li.bx._svp_item')
len(lis)

104

In [8]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, date, content

('갖고있는 장미 향수 비교',
 '4일 전',
 '로즈 향수들이 적네요 사실 다들 꽤나 유명한 향수들이라서 이미 알고 계신 분들이 많으시겠지만 혹시라도 비교해보고 싶으신 분이나 다른 사람에게 어떻게 느껴지는지 궁금하신 분도 계실 것 같아 글 적어보려 합니다...')

In [9]:
title_list, content_list, date_list = [],[],[]
for i, li in enumerate(lis):
    try:    #어디서 에러났는지 보려고 try, i변수, enumerate사용
        title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)
    except:
        print(i)

10
11
22
23


In [12]:
title_list, content_list, date_list = [],[],[]
for i, li in enumerate(lis):
    try:
        adv = li.select_one('a.link_ad').get_text()
        if adv.strip() == '광고':
            continue
    except:
        title = li.select_one('a.api_txt_lines.total_tit').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)

        #print(i)

In [14]:
df = pd.DataFrame({
    'title':title_list, 'date':date_list, 'content':content_list
})
df.head()

,title,date,content
0,갖고있는 장미 향수 비교,4일 전,로즈 향수들이 적네요 사실 다들 꽤나 유명한 향수들이라서 이미 알고 계신 분들이 많...
1,"옛새(YETSAE) 누드 인 포그, 나만 알고 싶은 젠더리스 니치 향수 브랜드",2021.08.14.,또 향수의 계절 아닙니까 ㅎ_ㅎ 여름엔 답답함이 느껴져 멀리했지만 요새 또 슬슬 분...
2,[남자 향수 추천] 품에 안기고 싶은 남자 향수,2021.08.03.,남자 향수 추천품에 안기고 싶은 남자 향수 추천 남자 향수 추천 : 품에 안기고 싶...
3,대니 맥켄지 밤쉘 디퓨저 후기 (나의 인생 향수),10시간 전,대니 맥켄지 밤쉘 디퓨저 후기 (나의 인생 향수) 안녕하세요 이웃님들 ! :) 오늘...
4,니치향수 SWG 고급짐 머선129?,2021.08.12.,예전에 얼핏 봤는데 많은 유럽인들은 향수를 쓰는 이유가 내 냄새를 가리기 위함이고 ...


In [15]:
df.shape

(100, 3)

In [16]:
df.to_csv('data/perfume_blog.txt')

In [17]:
driver.close()